## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [275]:
!pip install tensorflow-gpu



     |████████████████████████████████| 380.8MB 47kB/s 


In [0]:
import tensorflow as tf
tf.enable_eager_execution
tf.__version__


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [164]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('/gdrive/My Drive/Documents/R6InternalLab/prices.csv')

### Check all columns in the dataset

In [181]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data.drop("date", inplace = True, axis = 1)
data.drop("symbol", inplace = True, axis = 1)

In [183]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
#import copy
prices_df = data.head(1000)

In [185]:
prices_df.shape

(1000, 5)

In [186]:
prices_df["volume"] = prices_df["volume"]/1000000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [187]:
prices_df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X = prices_df[prices_df.columns[0:-1]]
y = prices_df["volume"]



X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size = 0.30)

In [189]:
X.shape

(1000, 4)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = np.array(X_train).astype('float32')
y_train = np.array(y_train).astype('float32')

In [0]:
X_test = np.array(X_test).astype('float32')
y_test = np.array(y_test).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
nm = Normalizer()
X_train =  nm.fit_transform(X_train)

In [0]:
from sklearn.preprocessing import Normalizer
X_test =  nm.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [194]:
X_train.shape

(700, 4)

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [199]:
for i in range(100):
    
    w, b = train(X_train, y_train, w, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)))

Current Loss on iteration 0 Tensor("Mean_403:0", shape=(), dtype=float32)
Current Loss on iteration 1 Tensor("Mean_405:0", shape=(), dtype=float32)
Current Loss on iteration 2 Tensor("Mean_407:0", shape=(), dtype=float32)
Current Loss on iteration 3 Tensor("Mean_409:0", shape=(), dtype=float32)
Current Loss on iteration 4 Tensor("Mean_411:0", shape=(), dtype=float32)
Current Loss on iteration 5 Tensor("Mean_413:0", shape=(), dtype=float32)
Current Loss on iteration 6 Tensor("Mean_415:0", shape=(), dtype=float32)
Current Loss on iteration 7 Tensor("Mean_417:0", shape=(), dtype=float32)
Current Loss on iteration 8 Tensor("Mean_419:0", shape=(), dtype=float32)
Current Loss on iteration 9 Tensor("Mean_421:0", shape=(), dtype=float32)
Current Loss on iteration 10 Tensor("Mean_423:0", shape=(), dtype=float32)
Current Loss on iteration 11 Tensor("Mean_425:0", shape=(), dtype=float32)
Current Loss on iteration 12 Tensor("Mean_427:0", shape=(), dtype=float32)
Current Loss on iteration 13 Tensor

### Get the shapes and values of W and b

In [200]:
print('Weights:\n', w)


Weights:
 Tensor("sub_1205:0", shape=(4, 1), dtype=float32)


In [201]:
print('Bias:\n',b)

Bias:
 Tensor("sub_1206:0", shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [202]:
X_test[0:1]

array([[-0.51530534, -0.4959871 , -0.4933617 , -0.49502668]],
      dtype=float32)

In [203]:
prediction(X_test[0:1], w, b)

<tf.Tensor 'Add_607:0' shape=(1, 1) dtype=float32>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_df = pd.read_csv('/gdrive/My Drive/Documents/R6InternalLab/Iris.csv')

In [239]:
iris_df.shape

(150, 6)

In [240]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
# preprocessing to convert the  target object type columns to numeric.
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing


dummy = pd.get_dummies(iris_df['Species'], prefix_sep ='_')

In [0]:
iris_df = pd.concat([iris_df,dummy], axis = 1)

In [243]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 9 columns):
Id                 150 non-null int64
SepalLengthCm      150 non-null float64
SepalWidthCm       150 non-null float64
PetalLengthCm      150 non-null float64
PetalWidthCm       150 non-null float64
Species            150 non-null object
Iris-setosa        150 non-null uint8
Iris-versicolor    150 non-null uint8
Iris-virginica     150 non-null uint8
dtypes: float64(4), int64(1), object(1), uint8(3)
memory usage: 7.6+ KB


In [0]:
iris_df.drop("Species", inplace = True, axis = 1)

In [0]:
iris_df.drop("Id", inplace = True, axis = 1)

### Splitting the data into feature set and target set

In [0]:
X_features = iris_df[iris_df.columns[0:4]]

In [247]:
X_features.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [0]:
Y_target = iris_df[iris_df.columns[4:7]]

In [249]:
Y_target.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_features, Y_target, test_size = 0.30)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [251]:
X_train_iris.shape

(105, 4)

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 1*4 to 784
#model.add(tf.keras.layers.Reshape((4,),input_shape=(4,)))
#model.add(tf.keras.layers.Reshape((784,),input_shape=(28,28,)))

#Normalize the data
#model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, input_shape=(4,), activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [258]:
y_test_iris.shape

(45, 3)

### Model Training 

In [259]:
model.fit(X_train_iris, y_train_iris, 
          validation_data=(X_test_iris, y_test_iris), 
          epochs=100,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 1s 11ms/sample - loss: 9.8701 - acc: 0.3429 - val_loss: 8.7820 - val_acc: 0.3111
Epoch 2/100
105/105 [==============================] - 0s 195us/sample - loss: 7.9075 - acc: 0.3429 - val_loss: 6.8173 - val_acc: 0.3111
Epoch 3/100
105/105 [==============================] - 0s 187us/sample - loss: 6.0109 - acc: 0.3429 - val_loss: 4.8457 - val_acc: 0.3111
Epoch 4/100
105/105 [==============================] - 0s 174us/sample - loss: 4.1884 - acc: 0.3429 - val_loss: 3.3189 - val_acc: 0.3111
Epoch 5/100
105/105 [==============================] - 0s 187us/sample - loss: 2.8537 - acc: 0.3143 - val_loss: 2.1751 - val_acc: 0.0000e+00
Epoch 6/100
105/105 [==============================] - 0s 174us/sample - loss: 2.0097 - acc: 0.0000e+00 - val_loss: 1.7751 - val_acc: 0.0000e+00
Epoch 7/100
105/105 [==============================] - 0s 194us/sample - loss: 1.7515 - acc: 0.0571 - val_loss: 1.5893 - v

### Model Prediction

In [260]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [271]:
 model.predict(X_test_iris)


array([[0.01841033, 0.4114952 , 0.57009447],
       [0.11527586, 0.47758296, 0.40714118],
       [0.8673047 , 0.1110481 , 0.02164722],
       [0.10490694, 0.45451206, 0.440581  ],
       [0.01797216, 0.3510977 , 0.6309301 ],
       [0.12872168, 0.47474095, 0.39653733],
       [0.74827325, 0.20289469, 0.04883197],
       [0.01906055, 0.34831494, 0.63262457],
       [0.85289216, 0.12794916, 0.01915866],
       [0.77344227, 0.18865374, 0.03790398],
       [0.8323937 , 0.13930704, 0.02829924],
       [0.01748811, 0.42622918, 0.55628276],
       [0.6733374 , 0.26165226, 0.06501035],
       [0.12827438, 0.5039741 , 0.36775154],
       [0.18272084, 0.48158723, 0.3356919 ],
       [0.02673583, 0.38241208, 0.5908521 ],
       [0.10865043, 0.4524046 , 0.43894494],
       [0.08186446, 0.5047965 , 0.41333908],
       [0.7751601 , 0.18283515, 0.04200479],
       [0.02122071, 0.3557388 , 0.62304044],
       [0.02057401, 0.31460142, 0.66482455],
       [0.12324074, 0.5154525 , 0.3613068 ],
       [0.

### Save the Model

In [0]:
model.save('iris_lir.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?